In [ ]:
# default_exp tasks_new

In [ ]:
#hide
from nbdev.showdoc import *

# Tasks class and its data management

## First Session 
#### **Condition:** If there are no entries in the 'essentials' for today in db

1. Ask **some** of the **essentials** and **5mj** Qs and write it on the db.

1. **Today's** task list compilation by reading todays entry on db.
    1. Compile a task list for today by:
         1. checking for any previously scheduled tasks in today's entry.
         2. reading the distraction journal and applying Task class.
         3. reading and checking if today is part of any of the reminders' scope.
         4. asking if any of the **actions** questions in 5mj are to coverted into task.

## Mid Session
1. Then continue running the app on **distraction journal** mode, if the app is closed and started any time but the first time, start this mode. Keep saving the entries persistently.
    > 
1. **Keep an option** in distraction mode, if you are about to sign out for the day, in that case, ask **day review** questions and rest of the 5mj questions.

## Last Session
#### **Condition:** From the distraction mode
1. During *last questions*, ask the 
   1. **essentials** and store it in that day's entry
   2. Ask rest of of 5mj questions
       1. Check if any of the strings from "actions" question had been converted into tasks and if yes, add a 'completed' tag.
       2. And store it in that day's entry
       
## I/O of Task and database classes
Task object will have following outputs: 
1. `.toDB` for database files.
1. `__str__` for app.
1. `.toJournal` for evernote.
1. `.toPhoto` for photo.

**STEPS**
1. Find "date(today)"
1. Goto "tasks" area
1. Find the tag for any scheduled tasks.
1. Ask user today's questions, referencing previous material by using dates as tags
1. Populate Tasks class when doing this. Write whatever data on the database.
1. "Day" class can handle lists of "Task" and others.

In [ ]:
#export
#from Journal.tasks import *
from Journal.questions import *
from datetime import datetime, date, time, timedelta
import re, math, json
from typing import *
from dataclasses import dataclass, field
# from yaml import load, dump
# try:
#     from yaml import CLoader as Loader, CDumper as Dumper
# except ImportError:
#     from yaml import Loader, Dumper

In [ ]:
#export
time_abbreviation = {'d':'days',
                     'h':'hours', 
                     'w':'weeks',
                     'm':'minutes'}

def str2timeDelta(string):
    
    pattern = re.compile(r"""
    (\d+\.*\d*)    # Any real numbers --> Digit decimal digit(optional)
    \s?            # To handle space
    ([hmsdw][o]?)  # Any of h, m... 'o' is to diff b/w m[in] and m[onth]
    """, re.VERBOSE)                         #REMEMBER: To add functionality for month later.
    args_as_tuples_in_list = re.findall(pattern, string)
    kwargs = {time_abbreviation.get(arg_tuple[1]) : float(arg_tuple[0]) 
              for arg_tuple in args_as_tuples_in_list}
    return timedelta(**kwargs)

In [ ]:
#export
def sensible_date2str(x: datetime.date) -> str:
    """
    Returns string of the form:
    '15-Dec-2012    [Saturday]' 
    """
#     if x.strftime("%d")[0] == '1': return x.strftime("%A %-dth %B %Y")
#     elif x.strftime("%d")[-1] == '1': return x.strftime("%A %-dst %B %Y")
#     elif x.strftime("%d")[-1] == '2': return x.strftime("%A %-dnd %B %Y")
#     elif x.strftime("%d")[-1] == '3': return x.strftime("%A %-drd %B %Y")
#     else : return x.strftime("%A %-dth %B %Y")
    return x.strftime("%d-%b-%Y    [%A]")
def sensible_str2date(x: str, case=0) -> datetime.date:
    """
    Takes string of the form:
    case 0 --> '15-Dec-2012' [default]
    case 1 --> '15-12-2012'
    """
    if case==0: x = datetime.strptime(x,'%d-%b-%Y')
    if case==1: x = datetime.strptime(x,'%d-%m-%Y')
    return x.date()

In [ ]:
#export 
def task_scheduler():
    """
    Ask the user for a date string and then convert it into datetime.date 
    object.
    """
    print(f"When should be this on your task list. \n")
    print(f"Press enter to add it on today's list. Today is {date.today()}")
    print(f"You can enter the date in the format of today's date printed above.")
    print(f'Or you can answer in relative terms. Eg: 10w, 5d (only weeks and days supported)')
    date_string = input()
    pattern_for_isodate = re.compile(r'\d{4}-\d{2}-\d{2}')
    
    if not date_string: # replaced if '' in date_string: , to prevent bug and also empty strings are falsy
        strt_Date = date.today()        
    elif re.match(pattern_for_isodate, date_string):
        strt_Date = date.fromisoformat(date_string)        
    else:
        strt_Date = date.today() + str2timeDelta(date_string)
        
    return strt_Date

In [ ]:
#export
def estimated_time():
    print(f'How much time do you think this would take ?')
    t = input("Eg: '45m', '1h'")
    estd_Time = str2timeDelta(t)
    return estd_Time

def actual_time(task_tuple):
    print(f'You thought you would take {task_tuple.Estd_Time}. How much time did you actually take ?')
    t = input("Eg: '45m', '1h'")
    task_tuple = task_tuple._replace( Time_Takn = str2timeDelta(t) )
    return task_tuple

def satisfaction(task_tuple):
    print("On a scale of 1-10, how much satisfied do you about the effort you put into this task ?")
    s = float(input())
    task_tuple = task_tuple._replace( Satsfactn = s)
    if task_tuple.completed == 1: 
        task_tuple = task_tuple._replace( CompDate = date.today() ) 
    return task_tuple


In [ ]:
@dataclass
class Task():    
    task: str
    day: int = 1
    estd_time = None
    actual_time: timedelta = timedelta(0)
    notes: str = None
    satisfaction_on_effort: int = 0
    prev_date: date = None
    next_date: date = date.today() + timedelta(days=1)
    completed: bool = False    
    if completed == True:
        satisfaction: int = 0        
            
    def __str__(self):        
        string = (f'{self.task}', 
                  f'Day = {self.day}    Complete = {self.completed}',
                  f'Satisfaction = {self.satisfaction_on_effort}',
                  f'Time\n    Estimated = {self.estd_time}',
                  f'\tActual = {self.actual_time}',
                  f'Date',
                  f"\t{(lambda x: f'Previous = {x}' if x != None else '')(self.prev_date)}",
                  f'\tNext = {self.next_date}',
                  f'Additional Notes:\n    {self.notes}')
        return '\n'.join(string)
    
    @classmethod
    def morning(cls):
        task = input('Enter the task\n')
        estd_time = input('How much time do you think this would take?\n')
        notes = input('Any additional notes you would like to add?\n')
        return cls(task=task, estd_time=estd_time, notes=notes)
    
 

@dataclass            
class Day():    
    date: date
    tasks: List['Task'] = field(default_factory=list)
    essentials: dict = field(default_factory=dict)
    mj: dict = field(default_factory=dict)
    
  #  def __post_init__():
        
    
    def toDB(self):
        with shelve.open('database') as database:
            database[self.date.isoformat()] = self

    @classmethod
    def fromDB(cls, date):
        with shelve.open('database') as db:
            d = db[date.isoformat()]            
        return cls(d.date, d.tasks, d.essentials, d.mj)

class Distraction():
    
    def save(*string):
        entries = {}
        try: entries = pickle.load(open('distraction.pkl', 'rb'))
        except FileNotFoundError: pass                                    
        finally:
            with open('distraction.pkl', 'wb') as f:
                _ = [entries.setdefault(i,datetime.datetime.now()) for i in string]
                pickle.dump(entries,f)

    def retrieve():
        try: entries = pickle.load(open('distraction.pkl', 'rb'))
        except FileNotFoundError: entries = {'--no-distractions--':None}
        return entries
    
    def delete(*string):
        entries = Distraction.retrieve()
        _ = [entries.pop(i) for i in string]
        with open('distraction.pkl', 'wb') as f: pickle.dump(entries,f)
        
    def delete_by_user_input():
        serial_no = input('Which one would you like to delete?\n')
        for index, key in enumerate(Distraction.retrieve()):
            if serial_no == index+1: break
        Distraction.delete(key)


In [ ]:
x = 'Comlete problem 7 on Hacker rank'
t = Task(x)
t.day[0].estd_time = str2timeDelta('1h')
t.day.append(str2timeDelta('15m'))

TypeError: 'int' object is not subscriptable

In [ ]:
ttt = TaskDtaBaseManagmntSystm(t);
ttt.getvars()

NameError: name 'TaskDtaBaseManagmntSystm' is not defined

In [ ]:
import math
class Circle(object):
    def __init__(self,radius,b):
        self.radius = radius
        self.b = b
    def area(self):
        return 3.1415 * self.radius**2,self.b
    
    def perimeter(self):
        return 2.0*math.pi*self.radius
    
    @classmethod
    def from_bbd(cls, bbd, b):
        radius = bbd/2.0/math.sqrt(2.0)
        return cls(radius,b)
    
    @staticmethod
    def angle_to_grade(angle):
        return math.tan(math.radians(angle))*100.0

In [ ]:
c = Circle.from_bbd(4,'.')
c.area()

In [ ]:
c = Circle.from_bbd(4)
d = Circle(4)
print(f"""
{c.area(), c.perimeter()}
{d.area(), d.perimeter()}
""")

In [ ]:
class Tire(Circle):
    def perimeter(self):
        return Circle.perimeter(self) * 1.25
t = Tire(4)
t.area(), t.perimeter()

In [ ]:
u = Tire.from_bbd(4)
u.area(), u.perimeter()

In [ ]:
a = Circle.angle_to_grade(4);a

In [ ]:
x = 'this is the ace cold'
print(f"Notes: {x.rjust(30)}")

In [ ]:
import datetime, pickle
from dataclasses import dataclass, field
entries = {}
distractions = ['str1', 'str2', 'str3']
for i in distractions:
    entries.setdefault(i, datetime.datetime.now())
entries

{'str1': datetime.datetime(2020, 7, 21, 6, 2, 35, 204231),
 'str2': datetime.datetime(2020, 7, 21, 6, 2, 35, 204240),
 'str3': datetime.datetime(2020, 7, 21, 6, 2, 35, 204243)}

In [ ]:
x  =datetime.datetime.now()

In [ ]:
with open('distraction.pkl','rb') as f:
    print(pickle.load(f))

{'str1': datetime.datetime(2020, 7, 20, 20, 41, 29, 227965), 'str2': datetime.datetime(2020, 7, 20, 20, 41, 29, 227974), 'str3': datetime.datetime(2020, 7, 20, 20, 41, 29, 227975)}


In [ ]:
Distraction.save('ab')

In [ ]:
Distraction.show()

['sample', 'sample', 'sample', 'ab', 'sample', 'ab', 'sample', 'ab']